In [21]:
import os
import platform
import configparser as cfg
from pyspark.sql import SparkSession,DataFrame, DataFrameNaFunctions
from pyspark.conf import SparkConf
from pyspark.sql.functions import col, when
import findspark
import pandas as pd
import numpy as np
configs = cfg.ConfigParser()
configs.read(str("C:\\Users\\pedam\\PycharmProjects\\pythonProject\\configs.properties"))

['C:\\Users\\pedam\\PycharmProjects\\pythonProject\\configs.properties']

In [2]:

def printc(input_text, in_clr):
    color = {'R': '\033[31m', 'G': '\033[32m', 'Y': '\033[33m', 'B': '\033[34m', 'P': '\033[35m', 'C': '\033[36m',
             'W': '\033[37m'}
    ENDC = '\033[m'
    print(color.get(in_clr) + ' ' + input_text + ' ' + ENDC)

    
def getSparkSession():
    spark = SparkSession.builder.config(conf=SparkConf()) \
        .master("local[*]") \
        .appName("Flatten_The_Nested_Json") \
        .getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    return spark



def setWIN_OS_Environment(config):
    hadoop_path = config.get('framework-variables', 'hadoop_path')
    os.environ["HADOOP_HOME"] = hadoop_path
    os.environ["hadoop.home.dir"] = hadoop_path
    os.environ["PATH"] = os.environ["PATH"] + f";{hadoop_path}\\bin"
    printc(os.environ["PATH"], 'G')
    findspark.init()


In [3]:
application_name = configs.get('framework-variables', 'application_name')
os_name = platform.system()
if os_name == 'Windows':
    setWIN_OS_Environment(configs)
spark = getSparkSession()

 C:\Users\pedam\anaconda3;C:\Users\pedam\anaconda3\Library\mingw-w64\bin;C:\Users\pedam\anaconda3\Library\usr\bin;C:\Users\pedam\anaconda3\Library\bin;C:\Users\pedam\anaconda3\Scripts;C:\Users\pedam\anaconda3\bin;C:\Users\pedam\anaconda3\condabin;C:\Users\pedam\anaconda3;C:\Users\pedam\anaconda3\Library\mingw-w64\bin;C:\Users\pedam\anaconda3\Library\usr\bin;C:\Users\pedam\anaconda3\Library\bin;C:\Users\pedam\anaconda3\Scripts;C:\Program Files\Common Files\Oracle\Java\javapath;C:\windows\system32;C:\windows;C:\windows\System32\Wbem;C:\windows\System32\WindowsPowerShell\v1.0;C:\windows\System32\OpenSSH;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\Users\Administrator\AppData\Local\Microsoft\WindowsApps;C:\Program Files\HP\OMEN-Broadcast\Common;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0;C:\WINDOWS\System32\OpenSSH;C:\Program Files\dotnet;C:\Program Files\nodejs;C:\Program

In [4]:
def filter_dataframe(df, columns_to_check):
    for column in columns_to_check:
        df = df.filter(
            ~col(column).isNull()
           # & ~col(column).rlike("[^a-zA-Z0-9_]")
        )
    return df




In [5]:

# Define the URL for the dataset
local_file = r'C:\Users\pedam\github_checkin_code\ml_learning_notebooks\src_data\Telecom_customer_churn.csv'

spark_df = spark.read.format("csv").option("header", "true").load(local_file)

spark_df.show(5)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

In [6]:
spark_df.count()

7043

In [7]:
spark_df = filter_dataframe(spark_df, spark_df.columns)

In [8]:
spark_df.count()

7043

In [9]:
# List of columns to map 'Yes' and 'No' to 1 and 0
columns_to_encode = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                  'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                  'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

In [10]:
# Map values using when and otherwise functions
for column in columns_to_encode:
    spark_df = spark_df.withColumn(
        column,
        when(col(column) == 'Yes', 1)
        .when(col(column) == 'No', 0)
        .when(col(column) == 'Male', 1)
        .when(col(column) == 'Female', 0)
        .when(col(column) == 'No phone service', 0)
        .when(col(column) == 'DSL', 0)
        .when(col(column) == 'Fiber optic', 1)
        .when(col(column) == 'Month-to-month', 1)
        .when(col(column) == 'One year', 2)
        .when(col(column) == 'Electronic check', 1)
        .when(col(column) == 'Mailed check', 2)
        .when(col(column) == 'Bank transfer (automatic)', 2)        
        .otherwise(col(column))
    )



In [11]:
# Show the updated DataFrame
spark_df.show(5,False)

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|7590-VHVEG|0     |0            |1      |0         |1     |0           |0            |0              |0             |1           |0               |0          |0          |0              |1       |1               |1

In [12]:
pandas_df = spark_df.toPandas()
pandas_df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,0,0,0,...,0,0,0,0,1,1,1,29.85,29.85,No
1,5575-GNVDE,1,0,0,0,34,1,0,0,1,...,1,0,0,0,2,0,2,56.95,1889.5,No
2,3668-QPYBK,1,0,0,0,2,1,0,0,1,...,0,0,0,0,1,1,2,53.85,108.15,Yes
3,7795-CFOCW,1,0,0,0,45,0,0,0,1,...,1,1,0,0,2,0,2,42.3,1840.75,No
4,9237-HQITU,0,0,0,0,2,1,0,1,0,...,0,0,0,0,1,1,1,70.7,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,1,1,24,1,1,0,1,...,1,1,1,1,2,1,2,84.8,1990.5,No
7039,2234-XADUH,0,0,1,1,72,1,1,1,0,...,1,0,1,1,2,1,Credit card (automatic),103.2,7362.9,No
7040,4801-JZAZL,0,0,1,1,11,0,0,0,1,...,0,0,0,0,1,1,1,29.6,346.45,No
7041,8361-LTMKD,1,1,1,0,4,1,1,1,0,...,0,0,0,0,1,1,2,74.4,306.6,Yes


In [13]:
new_pd_df = pd.get_dummies(pandas_df, columns=columns_to_encode, dummy_na=True)
new_pd_df

,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_0,gender_1,gender_nan,Partner_0,...,Contract_2,Contract_Two year,Contract_nan,PaperlessBilling_0,PaperlessBilling_1,PaperlessBilling_nan,PaymentMethod_1,PaymentMethod_2,PaymentMethod_Credit card (automatic),PaymentMethod_nan
0,7590-VHVEG,0,1,29.85,29.85,No,True,False,False,False,...,False,False,False,False,True,False,True,False,False,False
1,5575-GNVDE,0,34,56.95,1889.5,No,False,True,False,True,...,True,False,False,True,False,False,False,True,False,False
2,3668-QPYBK,0,2,53.85,108.15,Yes,False,True,False,True,...,False,False,False,False,True,False,False,True,False,False
3,7795-CFOCW,0,45,42.3,1840.75,No,False,True,False,True,...,True,False,False,True,False,False,False,True,False,False
4,9237-HQITU,0,2,70.7,151.65,Yes,True,False,False,True,...,False,False,False,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,0,24,84.8,1990.5,No,False,True,False,False,...,True,False,False,False,True,False,False,True,False,False
7039,2234-XADUH,0,72,103.2,7362.9,No,True,False,False,False,...,True,False,False,False,True,False,False,False,True,False
7040,4801-JZAZL,0,11,29.6,346.45,No,True,False,False,False,...,False,False,False,False,True,False,True,False,False,False
7041,8361-LTMKD,1,4,74.4,306.6,Yes,False,True,False,False,...,False,False,False,False,True,False,False,True,False,False


In [14]:

# Split the dataset into features (X) and target variable (y)
new_pd_df = new_pd_df.drop(['customerID'], axis=1)
X = new_pd_df.drop('Churn', axis=1)
y = new_pd_df['Churn']


In [15]:
new_pd_df

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_0,gender_1,gender_nan,Partner_0,Partner_1,...,Contract_2,Contract_Two year,Contract_nan,PaperlessBilling_0,PaperlessBilling_1,PaperlessBilling_nan,PaymentMethod_1,PaymentMethod_2,PaymentMethod_Credit card (automatic),PaymentMethod_nan
0,0,1,29.85,29.85,No,True,False,False,False,True,...,False,False,False,False,True,False,True,False,False,False
1,0,34,56.95,1889.5,No,False,True,False,True,False,...,True,False,False,True,False,False,False,True,False,False
2,0,2,53.85,108.15,Yes,False,True,False,True,False,...,False,False,False,False,True,False,False,True,False,False
3,0,45,42.3,1840.75,No,False,True,False,True,False,...,True,False,False,True,False,False,False,True,False,False
4,0,2,70.7,151.65,Yes,True,False,False,True,False,...,False,False,False,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,24,84.8,1990.5,No,False,True,False,False,True,...,True,False,False,False,True,False,False,True,False,False
7039,0,72,103.2,7362.9,No,True,False,False,False,True,...,True,False,False,False,True,False,False,False,True,False
7040,0,11,29.6,346.45,No,True,False,False,False,True,...,False,False,False,False,True,False,True,False,False,False
7041,1,4,74.4,306.6,Yes,False,True,False,False,True,...,False,False,False,False,True,False,False,True,False,False


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
print(new_pd_df.dtypes)
        
        

SeniorCitizen                              int64
tenure                                     int64
MonthlyCharges                           float64
TotalCharges                             float64
Churn                                     object
gender_0                                    bool
gender_1                                    bool
gender_nan                                  bool
Partner_0                                   bool
Partner_1                                   bool
Partner_nan                                 bool
Dependents_0                                bool
Dependents_1                                bool
Dependents_nan                              bool
PhoneService_0                              bool
PhoneService_1                              bool
PhoneService_nan                            bool
MultipleLines_0                             bool
MultipleLines_1                             bool
MultipleLines_nan                           bool
InternetService_0   

In [40]:
for column in new_pd_df.columns:
    if new_pd_df[column].dtype != 'bool': 
        new_pd_df[column] = new_pd_df[column].replace(' ', np.nan)
        new_pd_df[column] = pd.to_numeric(new_pd_df[column])
        unique_values = new_pd_df[column].unique()
        print(f"Unique values in {column}: {unique_values}")


Unique values in SeniorCitizen: [0 1]
Unique values in tenure: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
Unique values in MonthlyCharges: [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
Unique values in TotalCharges: [  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]


ValueError: Unable to parse string "No" at position 0

In [30]:


new_pd_df['TotalCharges'] = new_pd_df['TotalCharges'].replace(' ', np.nan)
new_pd_df['MonthlyCharges'] = new_pd_df['MonthlyCharges'].replace(' ', np.nan)


In [31]:
new_pd_df['TotalCharges'] = pd.to_numeric(new_pd_df['TotalCharges'])
new_pd_df['MonthlyCharges'] = pd.to_numeric(new_pd_df['MonthlyCharges'])

In [41]:
unique_values_total_charges = new_pd_df['TotalCharges'].unique()
unique_values_monthly_charges = new_pd_df['MonthlyCharges'].unique()

print(unique_values_total_charges)
pd.set_option('display.max_colwidth', None)

unique_values_monthly_charges = new_pd_df['MonthlyCharges'].unique()
print(f"Unique values in 'MonthlyCharges': {unique_values_monthly_charges}")

[  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Unique values in 'MonthlyCharges': [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]


In [43]:
# Build a Random Forest Classifier model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

ValueError: could not convert string to float: ' '

In [34]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(new_pd_df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

